In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Read in an image
import numpy as np
from PIL import Image

# Youll need to put an image here. I've got a picture of apples
img = Image.open("../../assets/test_imgs/apples.png").convert("L")
img.load()
img = np.asarray(img)

In [ ]:
# Resize the image to something better
from skimage.transform import resize
img = resize(img, (128, 128), anti_aliasing=True)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img, cmap="gray")
_ = plt.axis("off")

In [ ]:
# We'll artificially create some patches by cropping the image into smaller pieces
from skimage.util import view_as_windows

patch_size = (32, 32)
assert all(p == patch_size[0] for p in patch_size), "Patch size must be square"

patches = view_as_windows(img, patch_size, step=patch_size[0])

In [ ]:
def plot_grid(patches: list[list[np.ndarray]]) -> plt.Figure:
    fig, axes = plt.subplots(
        len(patches),
        len(patches[0]),
        figsize=(len(patches), len(patches[0])),
        gridspec_kw={"wspace": 0.02, "hspace": 0.02},
    )
    for axes_row, patches_row in zip(axes, patches, strict=True):
        for axis, patch in zip(axes_row, patches_row, strict=True):
            axis.imshow(patch, cmap="gray", vmin=0, vmax=1, aspect="auto")
            axis.axis("off")

    return fig


_ = plot_grid(patches)

In [ ]:
# Choose patches randomly
from current_denoising.generation import quilting

# I expect to pass a flat list of patches to my function
patches = patches.reshape(-1, 32, 32)

target_size = (128, 128)
patch_overlap = 4

rng = np.random.default_rng(0)

random_patch_grid = quilting.randomly_choose_patches(
    patches, target_size, patch_overlap, allow_rotation=False, rng=rng
)

In [ ]:
fig = plot_grid(random_patch_grid)
fig.suptitle("Randomly chosen patches")
fig.subplots_adjust(top=0.93)

In [ ]:
rng = np.random.default_rng(0)

optimal_patch_grid = quilting.optimally_choose_patches(
    patches, target_size, patch_overlap, allow_rotation=False, rng=rng
)

In [ ]:
fig = plot_grid(optimal_patch_grid)
fig.suptitle("Optimally chosen patches")
fig.subplots_adjust(top=0.93)

In [ ]:
rng = np.random.default_rng(0)
penalised_patch_grid = quilting.optimally_choose_patches(
    patches,
    target_size,
    patch_overlap,
    allow_rotation=False,
    rng=rng,
    repeat_penalty=1,
)
fig = plot_grid(penalised_patch_grid)
fig.suptitle("Optimally chosen patches, penalised for reuse")
fig.subplots_adjust(top=0.93)

In [ ]:
# We can join these naively,by just averaging the overlapping regions
naive_images = [
    quilting.naive_join_patches(patch_grid, patch_overlap, target_size)
    for patch_grid in [random_patch_grid, optimal_patch_grid, penalised_patch_grid]
]

In [ ]:
# It might be instructive to plot the FFTs
ffts = [(np.abs(np.fft.fft2(img_))) for img_ in [img] + naive_images]

# Shift the zero frequency component to the center
ffts = [np.fft.fftshift(fft) for fft in ffts]

In [ ]:
fig, axes = plt.subplots(
    2, 4, figsize=(15, 5), gridspec_kw={"wspace": 0.02, "hspace": 0.02}
)
for ax, image, title in zip(
    axes[0],
    [img] + naive_images,
    [
        "Original Image",
        "Randomly chosen patches",
        "Optimally chosen patches",
        "Penalised patches",
    ],
    strict=True,
):
    ax.imshow(image, cmap="gray")
    ax.set_title(title)

for ax, fft in zip(axes[1], ffts, strict=True):
    ax.imshow(np.log(fft + 1), cmap="gray")
    ax.axis("off")

In [ ]:
# It would actually be better to plot a histogram of gradients - sharp gradients indicate edges